# Introduction
After the previous commits in notebook 33, Ben noticed that there are some issues that arise that prevent the model from working in MatLab. Here I will fix those issues.

In [2]:
import cameo
import pandas as pd
import cobra.io

In [59]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

## Modify EC code in annotations
Here, the multiple EC codes are stored in the annotations as strings instead of as lists. This is a problem and will not be tolerated by CobraToolbox in Matlab and so needs to be fixed.

I also observed one metabolite still has 'EC ' in the annotation, I will write a script to remove it from whereever it is still present.

In [4]:
for rct in model.reactions:
    try:
        anno= rct.annotation['ec-code']
        if type(anno) == str:
            if len(anno) > 7:
                anno = anno.split(sep = ' ')
                rct.annotation['ec-code'] = anno
            else:
                continue
        else: 
            continue
    except KeyError: #rcts without ec-code
        continue

In [6]:
#remove any 'EC' in the annotation field
for rct in model.reactions:
    try:
        anno = rct.annotation['ec-code']
        for a in anno:
            if a in 'EC':
                anno.remove(a)
            else:
                continue
    except KeyError: #where there is no ec-code
        continue

Also, I see that there are some semicolons in the notes that were lifted to the annotation. I will remove them from the annotation field.

In [60]:
for rct in model.reactions:
    try:
        anno = rct.annotation['ec-code']
        if type(anno) == list:
            for ec in anno:
                if ec.find(';') < 0:
                    continue
                elif ec.find(';') > 0:
                    anno.remove(ec)
                    ec = ec.replace(';','')
                    anno.append(ec)
        if type(anno) == str:
            anno.replace(';','')
        else:
            continue
    except KeyError:#for rcts without ec code
        continue

In [64]:
#Save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Remove 'other group'
Per Ben's recommendation, I will remove the 'other' group that was added to include all reactions into a group.


In [68]:
model.groups.Other.members[0:5]

[<Reaction IDPh at 0x1cc23aa0d88>,
 <Reaction CCP at 0x1cc23aa1e48>,
 <Reaction HYDA at 0x1cc23aa2108>,
 <Reaction FOCYCTCOR at 0x1cc23aa23c8>,
 <Reaction FOCYTCCOR at 0x1cc23aa7f48>]

In [69]:
model.remove_groups(model.groups.Other)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:836: UserWarning:

need to pass in a list



In [71]:
model.groups

[<Group 00000 at 0x1cc2324db88>,
 <Group 00010 at 0x1cc2324d488>,
 <Group 00020 at 0x1cc23dafe88>,
 <Group 00030 at 0x1cc23dafd48>,
 <Group 00040 at 0x1cc23daf708>,
 <Group 00051 at 0x1cc2324da48>,
 <Group 00052 at 0x1cc23dafbc8>,
 <Group 00053 at 0x1cc23daf408>,
 <Group 00061 at 0x1cc23daf6c8>,
 <Group 00062 at 0x1cc23daf548>,
 <Group 00071 at 0x1cc23dafc08>,
 <Group 00072 at 0x1cc23daf788>,
 <Group 00073 at 0x1cc23daf148>,
 <Group 00100 at 0x1cc23daf288>,
 <Group 00120 at 0x1cc23daf388>,
 <Group 00121 at 0x1cc23daaac8>,
 <Group 00130 at 0x1cc23daae48>,
 <Group 00140 at 0x1cc23daa688>,
 <Group 00190 at 0x1cc23daaf08>,
 <Group 00195 at 0x1cc23daf048>,
 <Group 00196 at 0x1cc23daaa48>,
 <Group 00220 at 0x1cc23daa908>,
 <Group 00230 at 0x1cc23daa888>,
 <Group 00232 at 0x1cc23daa048>,
 <Group 00240 at 0x1cc23daa1c8>,
 <Group 00250 at 0x1cc23daa788>,
 <Group 00253 at 0x1cc23daa848>,
 <Group 00254 at 0x1cc23daa088>,
 <Group 00260 at 0x1cc23da5c48>,
 <Group 00261 at 0x1cc23daa808>,
 <Group 00